In [86]:
import requests
import json
import pandas as pd
import time
import re
pd.set_option('display.max_colwidth', None)

In [87]:
def get_tx_hash(events):
    for event in events:
        if(event['type'] == 'tx'):
            for attr in event['attributes']:
                if('composite_key' in attr and attr['composite_key']=='tx.hash'):
                    return attr['value']
    return None

In [88]:
def get_tx_action(events):
    for event in events:
        if(event['type'] == 'message'):
            action = None
            for attr in event['attributes']:
                if('key' in attr and attr['key']=='action'):
                    return attr['value']
    return None

In [89]:
def get_tx_contract_address(events):
    for event in events:
        if(event['type'] == 'execute'):
            action = None
            for attr in event['attributes']:
                if('key' in attr and attr['key']=='_contract_address'):
                    return attr['value']
    return None

In [90]:
def get_tx_coin_spent(event):
    spender = None
    amount = None
    if(event['type'] == 'coin_spent'):    
        for attr in event['attributes']:
            if('key' in attr and attr['key']=='spender'):
                spender = attr['value']
            if('key' in attr and attr['key']=='amount'):
                amount = attr['value']
    return spender, amount

In [91]:
def get_tx_coin_received(event):
    receiver = None
    amount = None
    if(event['type'] == 'coin_received'):
        for attr in event['attributes']:
            if('key' in attr and attr['key']=='receiver'):
                receiver = attr['value']
            if('key' in attr and attr['key']=='amount'):
                amount = attr['value']
    return receiver, amount

In [95]:
def parse_tx(tx):
    tx_parsed_tuples = []
    tx_created_at = tx['block']['created_at']
    tx_events = tx['events']
    tx_hash = get_tx_hash(tx_events)
    tx_action = get_tx_action(tx_events)
    tx_contract_address = get_tx_contract_address(tx_events)
    for event in tx_events:
        spender, amount = get_tx_coin_spent(event)
        if(spender and amount):
            tx_parsed_tuples.append((tx_created_at, tx_hash, spender, amount, event['type'], tx_action, tx_contract_address))
        receiver, amount = get_tx_coin_received(event)
        if(receiver and amount):
            tx_parsed_tuples.append((tx_created_at, tx_hash, receiver, amount, event['type'], tx_action, tx_contract_address))
    return tx_parsed_tuples

In [101]:
df = None
i = 0
limit = 10
raw_msgs = []
while True:
    # API endpoint URL
    address = 'kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu'
    url = f"https://api.kujira.app/api/txs?q={address}&limit={limit}&offset={i}&order_by=rowid&order_dir=desc"
    print(url)
    # Send a GET request to the API
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Retrieve the JSON data
        n_txs = len(response.json()['txs'])
        print(f"Received: {n_txs} txs")
        if(n_txs == 0):
            break
        
        json_data = response.json()
        raw_msgs.append(json_data)
        vals = []
        for tx in response.json()['txs']:
            # Call the parse_json function to parse the JSON data
            vals = vals + parse_tx(tx)
        if(df is None):
            df = pd.DataFrame(vals, columns=['created_at','tx_hash', 'address', 'amount','operation','action', 'contract_address'])
        else:
            df = pd.concat([df, pd.DataFrame(vals, columns=['created_at','tx_hash', 'address', 'amount','operation','action', 'contract_address'])])
        i += limit
        print(i)
        print(len(df))
    else:
        # Handle any errors that occurred during the request
        print("Error:", response.status_code)
df.to_csv(f'data/{address}.csv', index=False)

https://api.kujira.app/api/txs?q=kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu&limit=10&offset=0&order_by=rowid&order_dir=desc
Received: 10 txs
10
152
https://api.kujira.app/api/txs?q=kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu&limit=10&offset=10&order_by=rowid&order_dir=desc
Received: 10 txs
20
272
https://api.kujira.app/api/txs?q=kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu&limit=10&offset=20&order_by=rowid&order_dir=desc
Received: 10 txs
30
353
https://api.kujira.app/api/txs?q=kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu&limit=10&offset=30&order_by=rowid&order_dir=desc
Received: 10 txs
40
413
https://api.kujira.app/api/txs?q=kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu&limit=10&offset=40&order_by=rowid&order_dir=desc
Received: 10 txs
50
458
https://api.kujira.app/api/txs?q=kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu&limit=10&offset=50&order_by=rowid&order_dir=desc
Received: 10 txs
60
505
https://api.kujira.app/api/txs?q=kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu&limit=10&o

In [103]:
df.created_at = pd.to_datetime(df.created_at)

In [106]:
df.created_at.max()

Timestamp('2023-08-11 18:43:19+0000', tz='UTC')

In [84]:
df[(df.tx_hash=='7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316')\
    & (df.operation=='coin_spent')]

,tx_hash,address,amount,operation,action,contract_address
0,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira15064722pe9p7dqxcu8g560ufw9ckcs48e2u4ng,9920ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
2,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu,3093750ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
4,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1g2wpusvvy66x248vgnm8hwqu7fw5fswz3wrdfal0zwff0ezvmf4qynnqtf,3093750ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
6,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira14hj2tavq8fpesdwxxcu44rty3hh90vhujrvcmstl4zr3txmfvw9sl4e867,3587816ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
8,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira14hj2tavq8fpesdwxxcu44rty3hh90vhujrvcmstl4zr3txmfvw9sl4e867,5389ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
10,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1g2wpusvvy66x248vgnm8hwqu7fw5fswz3wrdfal0zwff0ezvmf4qynnqtf,3587816ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
12,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu,17939ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
14,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu,17939ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
16,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu,3551938ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu


In [94]:
df

,tx_hash,address,amount,operation,action,contract_address
0,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira15064722pe9p7dqxcu8g560ufw9ckcs48e2u4ng,9920ukuji,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
1,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira17xpfvakm2amg962yls6f84z3kell8c5lp3pcxh,9920ukuji,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
2,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu,3093750ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
3,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1g2wpusvvy66x248vgnm8hwqu7fw5fswz3wrdfal0zwff0ezvmf4qynnqtf,3093750ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
4,7DD3341D97655F2C6A4423757AD26EDAB89CD0FF0DAA507E32D8EBA08ABFF316,kujira1g2wpusvvy66x248vgnm8hwqu7fw5fswz3wrdfal0zwff0ezvmf4qynnqtf,3093750ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,kujira1e6fjnq7q20sh9cca76wdkfg69esha5zn53jjewrtjgm4nktk824stzyysu
...,...,...,...,...,...,...
0,59FF4DF16DEE76E53404AC5894AD8462FDEC8A2718592AFCAD6B33A99C8BE479,kujira1fegapd4jc3ejqeg0eu3jk4hvr74hg6600j2uf8,427ukuji,coin_spent,/ibc.core.client.v1.MsgUpdateClient,None
1,59FF4DF16DEE76E53404AC5894AD8462FDEC8A2718592AFCAD6B33A99C8BE479,kujira17xpfvakm2amg962yls6f84z3kell8c5lp3pcxh,427ukuji,coin_received,/ibc.core.client.v1.MsgUpdateClient,None
2,59FF4DF16DEE76E53404AC5894AD8462FDEC8A2718592AFCAD6B33A99C8BE479,kujira1yl6hdjhmkf37639730gffanpzndzdpmhlh536f,49000000ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F,coin_received,/ibc.core.client.v1.MsgUpdateClient,None
3,59FF4DF16DEE76E53404AC5894AD8462FDEC8A2718592AFCAD6B33A99C8BE479,kujira1yl6hdjhmkf37639730gffanpzndzdpmhlh536f,49000000ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F,coin_spent,/ibc.core.client.v1.MsgUpdateClient,None
